<a href="https://colab.research.google.com/github/Adityacse1001/Fake-News-Detection/blob/main/finaladitya.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tensorflow

In [3]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense

In [4]:
# Download NLTK data
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [5]:

# Load the datasets
true_data = pd.read_csv('/content/drive/MyDrive/True.csv')
fake_data = pd.read_csv('/content/drive/MyDrive/Fake.csv')

In [6]:

# Add a label column to indicate whether the news is true (1) or fake (0)
true_data['label'] = 1
fake_data['label'] = 0


In [7]:
# Concatenate the datasets
data = pd.concat([true_data, fake_data])

In [8]:
# Text Preprocessing Function
def preprocess_text(text):
    # Lowercasing
    text = text.lower()
    # Removing punctuation and non-alphabetic characters
    text = re.sub(r'[^a-z\s]', '', text)
    # Tokenizing
    tokens = text.split()
    # Removing stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    # Lemmatization
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)

    # Apply preprocessing to the text data
data['text'] = data['text'].apply(preprocess_text)



In [9]:
# Split the data into training and testing sets
train_data, test_data, train_labels, test_labels = train_test_split(data['text'], data['label'], test_size=0.2, random_state=42)


# Tokenize the text and convert it into sequences
tokenizer = Tokenizer()

tokenizer.fit_on_texts(train_data)

train_sequences = tokenizer.texts_to_sequences(train_data)
test_sequences = tokenizer.texts_to_sequences(test_data)

In [10]:
# Pad the sequences to have the same length
max_sequence_length = 1000  # choose an appropriate value
train_sequences = pad_sequences(train_sequences, maxlen=max_sequence_length)
test_sequences = pad_sequences(test_sequences, maxlen=max_sequence_length)

In [11]:
# Define the model
embedding_dim = 100  # choose an appropriate value

model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, embedding_dim, input_length=max_sequence_length))
model.add(LSTM(128))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [12]:
# Train the model
model.fit(train_sequences, train_labels, epochs=10, batch_size=32)

Epoch 1/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 55s 44ms/step - accuracy: 0.9246 - loss: 0.1946
Epoch 2/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 79s 44ms/step - accuracy: 0.9859 - loss: 0.0477
Epoch 3/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 82s 44ms/step - accuracy: 0.9946 - loss: 0.0175
Epoch 4/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 83s 45ms/step - accuracy: 0.9915 - loss: 0.0229
Epoch 5/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 82s 45ms/step - accuracy: 0.9973 - loss: 0.0090
Epoch 6/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 82s 45ms/step - accuracy: 0.9993 - loss: 0.0031
Epoch 7/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 81s 44ms/step - accuracy: 0.9999 - loss: 8.0934e-04
Epoch 8/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 50s 45ms/step - accuracy: 0.9999 - loss: 0.0011
Epoch 9/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 81s 44ms/step - accuracy: 0.9999 - loss: 7.4774e-04
Epoch 10/10
1123/1123 ━━━━━━━━━━━━━━━━━━━━ 82s 44ms/step - accuracy: 0.9999 - loss: 4.7771e-04


In [14]:
model.save("test.h5")

In [15]:
unseen_data=["In perhaps the first indication of what the New Zealand  inquiry committee, formed in the wake of an alleged plot to assassinate Khalistan separatist Gurpatwant Singh Pannun, could be looking at, US Deputy Secretary of State Kurt Campbell said Wednesday his country believes “Indian colleagues are looking carefully at what potential institutional reforms might be necessary in thewake of some of these allegations and reports. Responding to questions during an online press briefing, days after his India visit with US National Security Advisor Jake Sullivan, Campbell said, “We made clear that we seek accountability and we have held constructive dialogue with New Zealand on this topic and they have been responsive towards our concerns. We have consistently asked for updates. The issue has been raised directly with the Indian government at the highest levels of leadership."]
import tensorflow as tf
# Preprocess the unseen data
unseen_sequences = tokenizer.texts_to_sequences(unseen_data)
unseen_sequences = pad_sequences(unseen_sequences, maxlen=max_sequence_length)

# Load the saved model
loaded_model = tf.keras.models.load_model('test.h5')

# Make predictions on the unseen data
predictions = loaded_model.predict(unseen_sequences)

for i, prediction in enumerate(predictions):
    if prediction >= 0.5:
        print(f"Article {i+1}: **Real news**")
    else:
        print(f"Article {i+1}: **Fake news**")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
Article 1: **Fake news**


In [16]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

# Download the 'punkt' resource
nltk.download('punkt')

# Apply NLP techniques to the unseen data
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
unseen_data=["Iranian negotiators reportedly have made a last-ditch push for more concessions from the U.S. and five other world powers as talks on the fate of Iran's nuclear program come down to the final days before a crucial deadline.The New York Times reported late Sunday that Tehran had backed away from a tentative promise to ship a large portion of its uranium stockpile to Russia, where it could not be used as part of any future weapons program. Western officials insisted to the paper that the uranium did not have to be sent overseas, but could be disposed of in other ways.The new twist in the talks comes just two days before the deadline for both sides to agree on a framework for a permanent deal. The final deadline for a permanent deal would not arrive until the end of June.However, if Iran insists on keeping its uranium in the country, it would undermine a key argument made in favor of the deal by the Obama administration. The Times reports that if the uranium had gone to Russia, it would have been converted into fuel rods, which are difficult to use in nuclear weapons. It is not clear what would happen to the uranium if it remained in Iran."]
processed_data = []
for article in unseen_data:
    # Tokenize the article into words
    words = nltk.word_tokenize(article)

    # Remove stop words and apply lemmatization
    processed_words = [lemmatizer.lemmatize(word) for word in words if word.lower() not in stop_words]

    # Join the processed words back into a sentence
    processed_article = ' '.join(processed_words)

    # Add the processed article to the list
    processed_data.append(processed_article)

# Preprocess the processed data
unseen_sequences = tokenizer.texts_to_sequences(processed_data)
unseen_sequences = pad_sequences(unseen_sequences, maxlen=max_sequence_length)

# Make predictions on the preprocessed data
predictions = loaded_model.predict(unseen_sequences)

# Print the predictions
for i, prediction in enumerate(predictions):
    if prediction >= 0.5:
        print(f"Article {i+1}: **Real news**")
    else:
        print(f"Article {i+1}: **Fake news**")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
Article 1: **Fake news**


In [ ]:
!pip install SpeechRecognition pydub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.8/32.8 MB 34.8 MB/s eta 0:00:00


In [ ]:
# Load the audio file
audio_file = "/content/drive/MyDrive/record_out.wav"
with sr.AudioFile(audio_file) as source:
    # Read the audio data from the file
    audio_data = r.record(source)

    # Convert speech to text
    text = r.recognize_google(audio_data)

# Assign the recognized text to unseen_data
unseen_data = [text]
print(text)
import tensorflow as tf
# Preprocess the unseen data
unseen_sequences = tokenizer.texts_to_sequences(unseen_data)
unseen_sequences = pad_sequences(unseen_sequences, maxlen=max_sequence_length)

# Load the saved model
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/best.h5')

# Make predictions on the unseen data
predictions = loaded_model.predict(unseen_sequences)

for i, prediction in enumerate(predictions):
    if prediction >= 0.5:
        print(f"Article {i+1}: **Real news**")
    else:
        print(f"Article {i+1}: **Fake news**")


Trump raises concerns over members of urban community sporting more than zero times at Addison supporter that the troubling practice could affect the outcome of the election recalling Republican Presidential nominee Donald Trump Express strong concerned Friday that members of urban communities were voting more than 0 times 440 Nation quotes 2016 election results
1/1 [==============================] - 1s 544ms/step
Article 1: **Fake news**


In [ ]:
!pip install pytesseract # Install the pytesseract library
import pytesseract
!pip install --upgrade pytesseract
!apt-get install tesseract-ocr
!apt-get install tesseract-ocr-deu

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 45 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 1s (3,443 kB/s)
Selecting previously unselected package tesseract-ocr-eng.
(Reading database ... 121925 files and directories currently installed.)
Preparing to unpack .../tesseract-ocr-

In [ ]:
from PIL import Image
!which tesseract
# Load the news article screenshot
image_path = '/content/Screenshot 2024-06-11 085326.png'
img = Image.open(image_path)

# Extract text from the image using Tesseract OCR
extracted_text = pytesseract.image_to_string(img)

# process the extracted text
unseen_data=[extracted_text]

import tensorflow as tf
# Preprocess the unseen data
unseen_sequences = tokenizer.texts_to_sequences(unseen_data)
unseen_sequences = pad_sequences(unseen_sequences, maxlen=max_sequence_length)

# Load the saved model
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/best.h5')

# Make predictions on the unseen data
predictions = loaded_model.predict(unseen_sequences)

for i, prediction in enumerate(predictions):
    if prediction >= 0.5:
        print(f"Article {i+1}: **Real news**")
    else:
        print(f"Article {i+1}: **Fake news**")


/usr/bin/tesseract


FileNotFoundError: [Errno 2] No such file or directory: '/content/Screenshot 2024-06-11 085326.png'

In [ ]:
! pip install translate
!pip install googletrans==3.1.0a0

In [ ]:
from googletrans import Translator
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Create an instance of the Translator class
translator = Translator()

# Translate a text from one language to another
def translate_text(text, source_lang, target_lang):
    translation = translator.translate(text, src=source_lang, dest=target_lang)
    return translation.text

# Example usage
input_text = "केंद्रीय शिक्षा मंत्रालय के निर्देश के बाद मेडिकल प्रवेश परीक्षा नीट-यूजी में कथित अनियमितताओं की जांच रविवार को सीबीआई ने अपने हाथ में ले ली। इस बीच बिहार पुलिस की आर्थिक अपराध इकाई ने पांच और लोगों को गिरफ्तार किया, जिसके साथ ही कथित पेपर लीक मामले में गिरफ्तार लोगों की कुल संख्या बढ़कर 18 हो गई है। राष्ट्रीय परीक्षा एजेंसी (NTA) ने कदाचार का पता चलने के बाद 17 अभ्यर्थियों को परीक्षा में शामिल होने से रोक दिया। पांच मई को आयोजित हुई परीक्षा रद्द करने की मांग के बीच मंत्रालय के अधिकारियों ने सरकार का पिछला रुख दोहराया कि गड़बड़ी की घटनाएं स्थानीय और छिटपुट स्तर पर हुईं तथा उन लाखों परीक्षार्थियों के करियर को खतरे में डालना उचित नहीं है, जिन्होंने परीक्षा सही ढंग से उत्तीर्ण की।"
source_language = "hindi"
target_language = "english"
translated_text = translate_text(input_text, source_language, target_language)


unseen_data = [translated_text]

# Preprocess the unseen data
unseen_sequences = tokenizer.texts_to_sequences(unseen_data)
unseen_sequences = pad_sequences(unseen_sequences, maxlen=max_sequence_length)

# Load the saved model
loaded_model = tf.keras.models.load_model('/content/drive/MyDrive/best.h5')

# Make predictions on the unseen data
predictions = loaded_model.predict(unseen_sequences)

for i, prediction in enumerate(predictions):
    if prediction >= 0.5:
        print(f"Article {i+1}: **Real news**")
    else:
        print(f"Article {i+1}: **Fake news**")

